In [2]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import h5py
import os
from tqdm import tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')
import itertools
import numba

Populating the interactive namespace from numpy and matplotlib


/Share/home/chenxupeng/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
cd /Share/home/chenxupeng/projects/training

/Share/home/chenxupeng/projects/training


1024*10!(3628800)

# 主要思路
miRNA第一个，有值的位置之后就不再考虑了，接下来考虑miRNA位置为0的 <br>
把matrix矩阵转成每个位点有值的位置为1 <br>
0100100111 ->10进制

In [5]:
int('00101',2)

5

In [9]:
int(bin(5)[2:])

101

for i in range(1024):
    print bin(i)[2:]

#### 产生十个位置的 0 1 binary array

In [32]:
binaryarr = np.zeros([1024,10]).astype('int')
for t in range(1024):
    binaryarr[t] =  np.concatenate((np.repeat(0,10-len(bin( t )[2:])),np.array([bin(t)[2:][i] for i in range(len(bin( t )[2:]))]).astype('int')))
    

In [34]:
binaryarr.shape  #十进制下0-1023

(1024, 10)

In [44]:
rankind = np.array(list(itertools.permutations(np.arange(0,10))))

In [45]:
rankind

array([[0, 1, 2, ..., 7, 8, 9],
       [0, 1, 2, ..., 7, 9, 8],
       [0, 1, 2, ..., 8, 7, 9],
       ...,
       [9, 8, 7, ..., 1, 2, 0],
       [9, 8, 7, ..., 2, 0, 1],
       [9, 8, 7, ..., 2, 1, 0]])

In [38]:
#### 该步骤可以一次算一种情况的dict中的值，每个值表示该顺序下每种可能的样本行应该取第几个值

In [39]:
(binaryarr!=0).argmax(axis=1)

array([0, 9, 8, ..., 0, 0, 0])

In [54]:
nums = rankind.shape[0]
dictarr = np.zeros([1024,nums])
for i in tqdm(range(nums)):
    dictarr[:,i] = (binaryarr[:,rankind[i]]!=0).argmax(axis=1)

100%|██████████| 3628800/3628800 [04:18<00:00, 14031.40it/s]


In [53]:
rankind.shape

(3628800, 10)

In [52]:
np.math.factorial(10)

3628800

In [47]:
dictarr

array([[0., 0., 0., ..., 0., 0., 0.],
       [9., 8., 9., ..., 8., 9., 8.],
       [8., 9., 7., ..., 9., 7., 7.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [55]:
with h5py.File('dictforallmapping') as f:
    f.create_dataset('data',data=dictarr)

#### dictarr to int8

In [9]:
with h5py.File('dictforallmapping') as f:
    dictarr = f['data'][:]

In [10]:
dictarr_ = dictarr.astype('int8')

In [ ]:
with h5py.File('dictforallmappingint8') as f:
    f.create_dataset('data',data=dictarr_)

### 步骤二
每一个样本：每一行 转成01 转成十进制 np.where  然后变成1*3628800
每一行这么处理，完成一个样本所有情况的统计
可以建一个10*3628800的矩阵
每个10*1的记录该order下十个位置统计了多少次

最终10*3628800
这样不占内存


#### 1 sample矩阵 转0 1

#### 2样本每行转成10进制 找到1024行中的一个  以下步骤开始逐行处理

In [107]:
def two2ten(arr):
    return int(''.join(arr.astype('str')),2)

In [108]:
testarr[0]

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 0])

In [121]:
#找到dict 887 行的所有情况的值
np.unique(dictarr[two2ten(testarr[0].astype('str'))])

array([0., 1., 2.])

In [ ]:
from numba import jit, int32

@jit(int32(int32, int32))
def f(x, y):
    # A somewhat trivial example
    return x + y

In [145]:
from numba import jit, int8,int32

In [ ]:
@jit(int32(int32, int32))
def f(dictarr_,):
    # A somewhat trivial example
    return x + y

nums = 30000000
setran = 100000
3741  
两个小时

nums = 30000000
setran = 10000
3917

numba
nums = 30000000
setran = 100000
3741  
两个小时


In [7]:
# 10*3628800 arr 记录这个样本10种rna在各种情况下的数值
nums = 30000000
setran = 100000

testarr = np.array(10*np.random.random(nums*10)).astype('int').reshape(-1,10)
testarr[testarr !=0] =1

setrange = np.arange(0,setran )
dictarr_ = dictarr[:,setrange]
savearr = np.zeros([10,setran])
for i in tqdm(range(nums)):
    savearr[dictarr_[two2ten(testarr[i].astype('str'))].astype('int'),np.arange(0,setran)] +=1
    #update_arr(i)

ValueError: Not a dataset (not a dataset)

所有的： 18.43 
362880： 116
36288： 1152
numba加速
5？

In [144]:
2**32

4294967296

In [146]:

@numba.jit(int8(int8, int8,int8,int32))
def update_arr(zerosarr,dictarr_,testarr,num):
    zerosarr[dictarr_[two2ten(testarr[num].astype('str'))].astype('int'),np.arange(0,setran)] =1
    return zerosarr

In [150]:
nums = 30000000
setran = 100000
testarr = np.array(10*np.random.random(nums*10)).astype('int').reshape(-1,10)
testarr[testarr !=0] =1

setrange = np.arange(0,setran )
dictarr_ = dictarr[:,setrange]
savearr = np.zeros([10,setran])
arrayzeros = np.zeros([10,setran])

In [151]:
for i in tqdm(range(nums)):
    savearr+=update_arr(arrayzeros,dictarr_,testarr,i)








  0%|          | 0/30000000 [00:00<?, ?it/s]






  0%|          | 36/30000000 [00:00<23:38:08, 352.58it/s]






  0%|          | 71/30000000 [00:00<23:52:42, 348.99it/s]






  0%|          | 106/30000000 [00:00<23:52:33, 349.03it/s]






  0%|          | 142/30000000 [00:00<23:49:16, 349.83it/s]






  0%|          | 177/30000000 [00:00<23:52:51, 348.95it/s]






  0%|          | 213/30000000 [00:00<23:51:46, 349.22it/s]






  0%|          | 249/30000000 [00:00<23:50:15, 349.59it/s]






  0%|          | 284/30000000 [00:00<23:52:08, 349.12it/s]






  0%|          | 319/30000000 [00:00<23:53:53, 348.70it/s]






  0%|          | 354/30000000 [00:01<23:54:12, 348.62it/s]






  0%|          | 389/30000000 [00:01<23:55:12, 348.38it/s]






  0%|          | 425/30000000 [00:01<23:54:38, 348.51it/s]






  0%|          | 460/30000000 [00:01<23:55:30, 348.30it/s]






  0%|          | 495/30000000 [00:01<23:57:07, 347.91it/s]






  0%|          | 530/30000000 [00:

KeyboardInterrupt: 

In [112]:
40000000/19./3600./24

24.366471734892784

In [114]:
3628800/24./24.

6300.0

In [115]:
dictarr.shape

(1024, 3628800)

In [127]:
24*24/360.

1.6